In [6]:
import logging

from telegram import (
    Poll,
    ParseMode,
    KeyboardButton,
    KeyboardButtonPollType,
    ReplyKeyboardMarkup,
    ReplyKeyboardRemove,
    Update,
)
from telegram.ext import (
    Updater,
    CommandHandler,
    PollAnswerHandler,
    PollHandler,
    MessageHandler,
    Filters,
    CallbackContext,
)

In [ ]:
import requests
import datetime
import tzlocal

In [3]:
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO
)
logger = logging.getLogger(__name__)

In [ ]:
token_owm = '7128a67c5491f64f8e8a4852e3851877'
lat = '-23.54365'
lon = '-47.49862'

In [ ]:
def dados_clima(token, lat, lon):
    values = []
    r = requests.get(f'https://api.openweathermap.org/data/2.5/onecall?lat={lat}&lon={lon}&lang=pt_br&units=metric&exclude=minutely,hourly&appid={token}')
    for i in range(4, 7):
        values.append(r.json()['daily'][i]['weather'][0]['description'])
    return values

In [4]:
def start(update: Update, context: CallbackContext) -> None:
    """Inform user about what this bot can do"""
    update.message.reply_text(
        'Please select /poll to get a Poll, /quiz to get a Quiz or /preview'
        ' to generate a preview for your poll'
    )

In [5]:
def poll(update: Update, context: CallbackContext) -> None:
    """Envia uma poll predefinida"""
    questions = ["Sexta-Feira", "Sábado", "Domingo"]
    message = context.bot.send_poll(
        update.effective_chat.id,
        "Presença Ocupação Dandara",
        questions,
        is_anonymous=False,
        allows_multiple_answers=True,
    )
    # Save some info about the poll the bot_data for later use in receive_poll_answer
    payload = {
        message.poll.id: {
            "questions": questions,
            "message_id": message.message_id,
            "chat_id": update.effective_chat.id,
            "answers": 0,
        }
    }
    context.bot_data.update(payload)


def receive_poll_answer(update: Update, context: CallbackContext) -> None:
    """Efetua a votação dos usuários"""
    answer = update.poll_answer
    poll_id = answer.poll_id
    try:
        questions = context.bot_data[poll_id]["questions"]
    # this means this poll answer update is from an old poll, we can't do our answering then
    except KeyError:
        return
    selected_options = answer.option_ids
    answer_string = ""
    for question_id in selected_options:
        if question_id != selected_options[-1]:
            answer_string += questions[question_id] + " and "
        else:
            answer_string += questions[question_id]
    context.bot.send_message(
        context.bot_data[poll_id]["chat_id"],
        f"{update.effective_user.mention_html()} feels {answer_string}!",
        parse_mode=ParseMode.HTML,
    )
    context.bot_data[poll_id]["answers"] += 1
    # Close poll after three participants voted
    if context.bot_data[poll_id]["answers"] == 3:
        context.bot.stop_poll(
            context.bot_data[poll_id]["chat_id"], context.bot_data[poll_id]["message_id"]
        )

In [ ]:
def dados_clima(token, lat, lon):
    values = []
    r = requests.get(f'https://api.openweathermap.org/data/2.5/onecall?lat={lat}&lon={lon}&lang=pt_br&units=metric&exclude=minutely,hourly&appid={token}')
    for i in range(1, 4):
        values.append(r.json()['daily'][i]['weather'][0]['description'])
    return values